In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as pltim
from PIL import Image as im
import numpy as np
import os 
import tensorflow as tf
from random import sample

In [ ]:
#reading "Augmented" images and data adjustment
all_photos =  np.ndarray([2805, 28, 28, 1])#935*3, photo dims
all_photos_names =  []
print(all_photos.shape)

#reading Lines
for i, filename in enumerate(os.listdir("Line_TS")):   
    tmp = im.open("Line_TS/" + filename)
    tmp.thumbnail((28,28))
    tmp2 = np.array(tmp.convert('L'))
    all_photos[i] = tmp2.reshape((28,28,1))
    all_photos_names.append(filename)

#reading Ellipse
for i, filename in enumerate(os.listdir("Ellipse_TS")):
    tmp = im.open("Ellipse_TS/" + filename)
    tmp.thumbnail((28,28))
    tmp2 = np.array(tmp.convert('L'))
    all_photos[i+85] = tmp2.reshape((28,28,1))
    all_photos_names.append(filename)
    
#reading Diamonds
for i, filename in enumerate(os.listdir("Diamond_TS")):
    tmp = im.open("Diamond_TS/" + filename)
    tmp.thumbnail((28,28))
    tmp2 = np.array(tmp.convert('L'))
    all_photos[i+170] = tmp2.reshape((28,28,1))
    all_photos_names.append(filename)

    
print(len(all_photos_names))
img2 = im.open("Line_TS/" + all_photos_names[0])
img2.show()
print(all_photos[0].shape)
print(type(all_photos[0]))

img2 = im.open("Ellipse_TS/" + all_photos_names[935])
img2.show()
print(all_photos[935].shape)
print(type(all_photos[935]))

img2 = im.open("Diamond_TS/" + all_photos_names[1870])
img2.show()  
print(all_photos[1870].shape)
print(type(all_photos[1870]))

In [ ]:
#flatten them
all_photosF = np.ndarray([2805, 784])#2805, 28*28*1
for j, imgg in enumerate(all_photos):
    all_photosF[j] = (imgg.flatten())
    
print(len(all_photosF))

In [ ]:
#labels(ground_truth) as one hot key
all_gt =  np.zeros([2805, 3])

for i in range(2805):
    if i > 1869:
        all_gt[i][2] = 1
    elif i > 934:
        all_gt[i][1] = 1
    else:
        all_gt[i][0] = 1

# print(all_gt[2800])
print(all_gt[1870:2620])


In [ ]:
#Training data(2250) 750*3, Testing data(555) 185*3
#750 1500 2250 , 185 370 555

X_tr = np.ndarray([2250,784])
y_tr = np.ndarray([2250,3])

X_v = np.ndarray([555,784])
y_v = np.ndarray([555,3])

#================Line(all_photosF[0->934]) 
indexes = sample(range(0,935), 750)
#print(len(indexes))
# print( len(np.delete(all_photosF[1870:2805],indexes,axis=0)))

X_tr[0:750] = all_photosF[indexes]
y_tr[0:750] = all_gt[0:750]

X_v[0:185] = np.delete(all_photosF[0:935],indexes,axis=0)
y_v[0:185] = all_gt[0:185]

#================Ellipse(all_photosF[935->1869])
indexes = sample(range(0,935), 750)

X_tr[750:1500] = all_photosF[indexes] 
y_tr[750:1500] = all_gt[935:1685]

X_v[185:370] = np.delete(all_photosF[935:1870],indexes,axis=0)
y_v[185:370] = all_gt[935:1120]

#================Diamonds(all_photosF[1870->2804])
indexes = sample(range(0,935), 750)

X_tr[1500:2250] = all_photosF[indexes]
y_tr[1500:2250] = all_gt[1870:2620]

X_v[370:555] = np.delete(all_photosF[1870:2805],indexes,axis=0)
y_v[370:555] = all_gt[1870:2055]

# print(y_tr.shape)
# # print(y_tr[0:210])

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

keep_prob = tf.placeholder(tf.float32)

In [ ]:
#Inputs
x = tf.placeholder(tf.float32, shape=[None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

#Gorund truth
y_ = tf.placeholder(tf.float32, [None, 3])

In [ ]:
#--Layer1
#Conv, Batch_Noram, Relu, DropOut
#kda eldropout tmam !!
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
batch_norm_conv1 = tf.contrib.layers.batch_norm(conv2d(x_image, W_conv1) + b_conv1)
h_conv1 = tf.nn.relu(batch_norm_conv1)
# h_conv1_drop = tf.nn.dropout(h_conv1, 0.5)
#h_pool1 = max_pool_2x2(h_conv1)

#--Layer2
W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])
batch_norm_conv2 = tf.contrib.layers.batch_norm( conv2d(h_conv1, W_conv2) + b_conv2 )
h_conv2 = tf.nn.relu( batch_norm_conv2)
# h_conv2_drop = tf.nn.dropout(h_conv2, 0.8)
# print (h_conv2_drop.shape)

#--fc1
W_fc1 = weight_variable([28 * 28 * 64, 256])
b_fc1 = bias_variable([256])
h_conv2_flat = tf.reshape(h_conv2, [-1, 28*28*64])
batch_norm_fc1 = tf.contrib.layers.batch_norm( tf.matmul(h_conv2_flat, W_fc1) + b_fc1 )
h_fc1 = tf.nn.relu(batch_norm_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
print (h_conv2_flat.shape)
print(W_fc1)

#--fc2
W_fc2 = weight_variable([256, 64])
b_fc2 = bias_variable([64])
batch_norm_fc2 = tf.contrib.layers.batch_norm( tf.matmul(h_fc1_drop, W_fc2)  + b_fc2 )
h_fc2 = tf.nn.relu(batch_norm_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#Readout layer
W_fc3 = weight_variable([64, 3])
b_fc3 = bias_variable([3])

y_conv = tf.matmul(h_fc2_drop, W_fc3) + b_fc3

In [ ]:
#Evaluate error
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
#Update params
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
#compare with the ground truth
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
from time import time

tic = time()

for i in range(5000):
    
    for j in range(0, 2250, 75):
        train_accuracy = accuracy.eval(feed_dict={ x: X_tr[j:j+75], y_: y_tr[j:j+75], keep_prob: 1.0})
        train_step.run(feed_dict={x: X_tr[j:j+75], y_: y_tr[j:j+75], keep_prob: 0.5})

    if i % 100 == 0:
        print('step %d, training accuracy %g' % (i, train_accuracy))
        

toc = time()
print('take time: %s' %((toc - tic)/60))

In [ ]:
#Evaluating !!!!!!!
print('test accuracy %g' % accuracy.eval(feed_dict={x: X_v, y_: y_v, keep_prob: 1.0}))
# print('test accuracy %g' % accuracy.eval(feed_dict={x: X_v, y_: y_v}))
writer = tf.summary.FileWriter('./Model_Summary' , sess.graph)

In [ ]:
#Saving and creating checkpoints
saver = tf.train.Saver()
save_path = saver.save(sess, "chkpts/model_1Cnn_aug_batch_drop.ckpt")

In [ ]:
#Restoring variables
saver = tf.train.Saver()
saver.restore(sess, "chkpts/model_1Cnn_aug_batch_drop.ckpt")
print("Model restored.")

In [ ]:
# momken ba2a ba3d ama a-restore el model a3mel test ezai ?!
print('test accuracy %g' % accuracy.eval(feed_dict={x: X_v, y_: y_v}))

In [ ]:
from IPython.display import Audio
sound_file = './sound/finish.mp3'
Audio(url=sound_file, autoplay=True)